# Project 5
# Car price prediction model

## Подготовка

Загружаем необходимые библиотеки, настраиваем отображение, инициализируем глобальные переменные

In [1]:
# imports
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import json
# options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# global definitions
RANDOM_STATE = 73

## 1. Анализ данных, которые необходимо получить

Прежде, чем начать мучить сайт auto.ru запросами, необходимо понять, какие данные и в каком виде необходимо получить.
Для этого загрузим тестовый датасет (по которому надо будет делать сабмишн) и немного поработаем с ним.

In [2]:
df_test = pd.read_csv('test.csv')

Сразу заменим имена столбцов на PEP-friendly

In [3]:
df_test.columns = ['body_type', 'brand', 'car_url', 'color', 'complectation',
              'description', 'engine_vol', 'engine_power', 'equipment',
              'fuel_type', 'image', 'mileage', 'model_date', 'model_info', 'model_name',
              'name', 'doors_num', 'parsing_unixtime', 'price_currency',
              'production_date', 'sell_id', 'super_gen', 'configuration',
              'transmission', 'vendor', 'num_owners', 'ownership', 'docs',
              'drive', 'steering', 'state', 'customs']

И посмотрим на сам дата-сет

In [4]:
df_test.sample(10)

,body_type,brand,car_url,color,complectation,description,engine_vol,engine_power,equipment,fuel_type,image,mileage,model_date,model_info,model_name,name,doors_num,parsing_unixtime,price_currency,production_date,sell_id,super_gen,configuration,transmission,vendor,num_owners,ownership,docs,drive,steering,state,customs
30866,пикап двойная кабина,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,чёрный,NaN,Авто в отличном состоянии. Причина продажи мех...,2.0 LTR,180 N12,"{""engine-proof"":true,""cruise-control"":true,""as...",дизель,https://autoru.naydex.net/cSRr96A18/708ec0RhY5...,265000,2010,"{""code"":""AMAROK"",""name"":""Amarok"",""ru_name"":""Ам...",AMAROK,full-time 2.0d MT (180 л.с.) 4WD,4,1603578589,RUB,2013,1095340742,"{""id"":""9263549"",""nameplate"":""full-time"",""displ...",PICKUP_TWO MECHANICAL 2.0,механическая,EUROPEAN,2 владельца,6 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен
10382,внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x1/11009238...,оранжевый,NaN,Недостатков нет.,2.0 LTR,184 N12,NaN,бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,87000,2012,"{""code"":""X1"",""name"":""X1"",""ru_name"":""Х1"",""morph...",X1,20i 2.0 AT (184 л.с.) 4WD,5,1603118838,RUB,2013,1100923838,"{""id"":""8247044"",""name"":""20i"",""nameplate"":""20i""...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
17703,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/cls_kl...,бежевый,"{""id"":""2442610"",""name"":""350"",""available_option...",Автомобиль в наличии. \nДва ключа.\nОригинальн...,3.5 LTR,272 N12,"{""cruise-control"":true,""esp"":true,""airbag-driv...",бензин,https://autoru.naydex.net/ESooa9149/468b9eoMem...,167000,2004,"{""code"":""CLS_KLASSE"",""name"":""CLS"",""ru_name"":""Ц...",CLS_KLASSE,350 3.5 AT (272 л.с.),4,1603248832,RUB,2007,1101266967,"{""id"":""2307782"",""name"":""350"",""nameplate"":""350""...",SEDAN AUTOMATIC 3.5,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен
29280,внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x6/11013407...,красный,"{""id"":""20593773"",""name"":""xDrive30d Luxury"",""av...",Автомобиль продаётся официальным дилером BMW -...,3.0 LTR,249 N12,"{""cruise-control"":true,""esp"":true,""adaptive-li...",дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,71481,2014,"{""code"":""X6"",""name"":""X6"",""ru_name"":""Х6"",""morph...",X6,30d 3.0d AT (249 л.с.) 4WD,5,1603559474,RUB,2014,1101340773,"{""id"":""20158772"",""name"":""30d"",""nameplate"":""30d...",ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
29370,внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x3/11013032...,синий,NaN,РОЛЬФ Премиум – крупнейший официальный дилер М...,2.0 LTR,245 N12,"{""asr"":true,""airbag-driver"":true,""aux"":true,""u...",бензин,https://autoru.naydex.net/f1bm7A798/f15c727Yw/...,48814,2014,"{""code"":""X3"",""name"":""X3"",""ru_name"":""Х3"",""morph...",X3,28i xDrive 2.0 AT (245 л.с.) 4WD,5,1603559951,RUB,2017,1101303242,"{""id"":""20156808"",""name"":""28i"",""nameplate"":""28i...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
11692,внедорожник 5 дв.,NISSAN,https://auto.ru/cars/used/sale/nissan/pathfind...,чёрный,NaN,"Штраф оплачен, юр ограничения будут сняты в бл...",2.5 LTR,174 N12,"{""electro-window-back"":true,""esp"":true,""airbag...",дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,197000,2004,"{""code"":""PATHFINDER"",""name"":""Pathfinder"",""ru_n...",PATHFINDER,2.5d MT (174 л.с.) 4WD,5,1603271807,RUB,2005,1101303595,"{""id"":""2308185"",""displacement"":2488,""engine_ty...",ALLROAD_5_DOORS MECHANICAL 2.5,механическая,JAPANESE,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
28863,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/paje..

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   body_type         34686 non-null  object
 1   brand             34686 non-null  object
 2   car_url           34686 non-null  object
 3   color             34686 non-null  object
 4   complectation     6418 non-null   object
 5   description       34686 non-null  object
 6   engine_vol        34686 non-null  object
 7   engine_power      34686 non-null  object
 8   equipment         24690 non-null  object
 9   fuel_type         34686 non-null  object
 10  image             34686 non-null  object
 11  mileage           34686 non-null  int64 
 12  model_date        34686 non-null  int64 
 13  model_info        34686 non-null  object
 14  model_name        34686 non-null  object
 15  name              34686 non-null  object
 16  doors_num         34686 non-null  int64 
 17  parsing_unix

В некоторых столбцах есть пропуски (о них чуть позже), много строк, мало цифр - придется попотеть(

### 1.1 Очистка от мусора

Посмотрим на уникальные значения в каждом столбце:

In [6]:
df_test.nunique().sort_values()

customs                 1
price_currency          1
state                   1
steering                2
docs                    2
vendor                  2
drive                   3
num_owners              3
transmission            4
fuel_type               5
doors_num               5
brand                  12
color                  16
body_type              24
engine_vol             55
model_date             66
production_date        69
ownership             275
engine_power          315
model_name            544
configuration         634
model_info            954
complectation        2364
name                 2780
super_gen            5890
mileage             11268
equipment           23705
description         31732
image               34557
parsing_unixtime    34686
car_url             34686
sell_id             34686
dtype: int64

3 столбца содержат всего по 1 уникальному значению, т.е. их наличие никак не влияет на модель. Взглянем на них:

In [7]:
df_test.head(1)[['customs', 'price_currency', 'state']]

,customs,price_currency,state
0,Растаможен,RUB,Не требует ремонта


Искать и парсить будем только объявления с растаможенными, не требующими ремонта автомобилями. цены будем получать в рублях.
Эти столбцы можно удалить. Также сразу удалим отсальные столбцы, не несущие полезной для модели информации.

In [8]:
df_test.drop(columns=['customs', 'price_currency', 'state', 'sell_id', 'car_url', 'image'], inplace=True)

### 1.2 Бинарные столбцы
Теперь разберемся со столбцами, в которых по 2-3 уникальных значения (т.е. по сути с категоральными столбцами)

In [9]:
df_test.steering.unique()

array(['Левый', 'Правый'], dtype=object)

In [10]:
# превратим категоральный признак в бинарный, проставив 1 везде, где руль справа
df_test.steering = (df_test.steering == 'Правый').astype('int')

In [11]:
df_test.docs.unique()

array(['Оригинал', 'Дубликат', nan], dtype=object)

In [12]:
# найдем наиболее частое значение
docs_fill = df_test.docs.value_counts().index[0] 
# заполним им все пропуски
# а затем превратим категоральный признак в бинарный, проставив 1 везде, где ПТС в виде дубликата
df_test.docs = (df_test.docs.fillna(docs_fill) == 'Дубликат').astype('int')

In [13]:
df_test.vendor.unique()

array(['EUROPEAN', 'JAPANESE'], dtype=object)

In [14]:
# затем превратим категоральный признак в бинарный, проставив 1 везде, где производитель из Японии
df_test.vendor = (df_test.vendor == 'JAPANESE').astype('int')

### 1.3 Категоральные столбцы

In [15]:
df_test.drive.unique()

array(['передний', 'полный', 'задний'], dtype=object)

Со столбцом `drive` пока ничего не будем делать

In [16]:
df_test.num_owners.unique()

array(['3 или более', '1\xa0владелец', '2\xa0владельца'], dtype=object)

По сути это числовой столбец, из которого достаточно взять исключительно первый символ

In [17]:
df_test.num_owners = df_test.num_owners.str[0].astype('int')

### 1.4 Составные столбцы

Для анализа составных столбцов добавим вспомогательные функции

In [18]:
def get_word(column, index=-1):
    return column.apply(lambda s: s.split()[index]) if index != -1 else column

def print_unique(column, index=-1):
    col_unique = get_word(column, index).unique()
    print(len(col_unique))
    print(col_unique)

Теперь разберем столбец `configuration`

In [19]:
print_unique(df_test.configuration, 0)

24
['LIFTBACK' 'ALLROAD_5_DOORS' 'HATCHBACK_5_DOORS' 'SEDAN' 'COMPACTVAN'
 'WAGON_5_DOORS' 'PICKUP_ONE' 'HATCHBACK_3_DOORS' 'COUPE' 'CABRIO'
 'MINIVAN' 'PICKUP_TWO' 'ALLROAD_3_DOORS' 'ROADSTER' 'MICROVAN'
 'SEDAN_2_DOORS' 'COUPE_HARDTOP' 'FASTBACK' 'TARGA' 'ALLROAD_OPEN'
 'LIMOUSINE' 'PICKUP_ONE_HALF' 'SEDAN_HARDTOP' 'VAN']


In [20]:
print_unique(df_test.body_type)

24
['лифтбек' 'внедорожник 5 дв.' 'хэтчбек 5 дв.' 'седан' 'компактвэн'
 'универсал 5 дв.' 'пикап одинарная кабина' 'хэтчбек 3 дв.' 'купе'
 'кабриолет' 'минивэн' 'пикап двойная кабина' 'внедорожник 3 дв.'
 'родстер' 'микровэн' 'седан 2 дв.' 'купе-хардтоп' 'фастбек' 'тарга'
 'внедорожник открытый' 'лимузин' 'пикап полуторная кабина'
 'седан-хардтоп' 'фургон']


In [21]:
print_unique(df_test.configuration, 1)

4
['ROBOT' 'MECHANICAL' 'AUTOMATIC' 'VARIATOR']


In [22]:
print_unique(df_test.transmission)

4
['роботизированная' 'механическая' 'автоматическая' 'вариатор']


In [23]:
print_unique(df_test.engine_vol, 0)

55
['1.2' '1.6' '1.8' '2.0' '1.4' '1.3' '1.0' '3.6' '1.5' '1.9' '2.8' '1.1'
 '2.5' '4.2' '3.0' '4.0' '5.9' '2.7' '3.1' '2.4' '5.2' '3.2' '4.1' '6.3'
 '2.3' '6.0' '2.2' '3.7' '2.9' '5.0' '3.3' '2.1' '2.6' 'LTR' '3.5' '1.7'
 '0.7' '4.4' '4.8' '5.4' '6.6' '4.9' '3.8' '3.4' '3.9' '4.6' '5.6' '4.5'
 '5.5' '6.2' '4.7' '4.3' '5.8' '5.3' '5.7']


Первое слово внутри столбца полностью соответствует столбцу `body_type`, а второе - столбцу `transmission`, а третье - вроде как соответствует столбцу `engine_vol`, который, в свою очередь сам составной. такое дублирование не нужно, исправим

In [24]:
# в столбцах body_type и transmission сохраним более привлекательные латинские значения
df_test.body_type = get_word(df_test.configuration, 0)
df_test.transmission = get_word(df_test.configuration, 1)
# в столбца engine_vol избавимся от 'LTR' и превратим значение в число
df_test.engine_vol = df_test.engine_vol.apply(lambda s: '0.0' if s == ' LTR' else s.split()[0]).astype('float')
df_test.drop(columns='configuration', inplace=True)

In [25]:
df_test.head()

,body_type,brand,color,complectation,description,engine_vol,engine_power,equipment,fuel_type,mileage,model_date,model_info,model_name,name,doors_num,parsing_unixtime,production_date,super_gen,transmission,vendor,num_owners,ownership,docs,drive,steering
0,LIFTBACK,SKODA,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,2014,"{""id"":""10373605"",""displacement"":1197,""engine_t...",ROBOT,0,3,NaN,0,передний,0
1,LIFTBACK,SKODA,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,2017,"{""id"":""20913311"",""displacement"":1598,""engine_t...",MECHANICAL,0,1,NaN,0,передний,0
2,LIFTBACK,SKODA,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,88000,2013,"{""code"":""SUPERB"",""name"":""Superb"",""ru_name"":""Су...",SUPERB,DSG 1.8 AMT (152 л.с.),5,1603226280,2014,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",ROBOT,0,1,NaN,0,передний,0
3,LIFTBACK,SKODA,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,95000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 AT (110 л.с.),5,1603226284,2014,"{""id"":""20105521"",""displacement"":1598,""engine_t...",AUTOMATIC,0,1,NaN,0,передний,0
4,LIFTBACK,SKODA,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,58536,2008,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.8 AT (152 л.с.),5,1603226288,2012,"{""id"":""4561004"",""displacement"":1798,""engine_ty...",AUTOMATIC,0,1,NaN,0,передний,0


Теперь столбец `engine_power`. Похоже, что в нем второе "слово" не имеет смысловой нагрузки для модели

In [26]:
print_unique(df_test.engine_power, 1)

1
['N12']


Так и есть. А первое "слово" также присутствует в столбце `name`, который в свою очередь, также дублирует столбцы `engine_vol` и `transmission`. Исправляем:

In [27]:
df_test.engine_power = get_word(df_test.engine_power, 0).astype('int')
df_test.drop(columns='name', inplace=True)

### 1.5 Прочие столбцы

Столбец `ownership` содержит срок владения. пропусков много, их оставим для следующих этапов, а имеющиеся данные распарсим и превратим в число (дробное кол-во лет)

In [28]:
months = df_test.ownership.str.extract(r'(\d{1,2})\sмесяц').astype('float')
years = df_test.ownership.str.extract(r'(\d{1,2})\s(?:лет|год)').astype('float')

df_test.ownership = np.round(years + (months / 12), 3)

Посмотрим на столбцы `model_info` и `super_gen`, данные в которых де-факто - словари

In [29]:
print(df_test.sample().model_info.iloc[0])

{"code":"A5","name":"A5","ru_name":"А5","morphology":{},"nameplate":{"code":"","name":"","semantic_url":""}}


In [30]:
print(df_test.sample().super_gen.iloc[0])

{"id":"9290826","displacement":1395,"engine_type":"GASOLINE","gear_type":"FORWARD_CONTROL","transmission":"ROBOT","power":122,"power_kvt":90,"human_name":"1.4 AMT (122 л.с.)","acceleration":9.3,"clearance_min":160,"fuel_rate":5}


Столбец `model_info` не несет новой информации, просто расширяет имеющийся столбец `model_name`, поэтому от него можно избавиться.

Столбец `super_gen` тоже не несет особо новой информации, дублируя столбцы `engine_vol`, `engine_power` т.д., однако для столбцов `fuel_type` и `drive` мы можем извлечь наименования, которые больше походят на технические (латинница, подчеркивания вместо пробелов и т.п.). Преобразуем эти столбцы.

In [31]:
def process_super_gen(row):
    sg = json.loads(row.super_gen)
    return [sg['engine_type'], sg['gear_type']]
    
df_test[['fuel_type', 'drive']] = df_test.apply(process_super_gen, axis=1, result_type='expand')
df_test.drop(columns=['model_info', 'super_gen'], inplace=True)

На текущий момент я не больше не вижу возможности что-либо сделать с  дата-сетом, не видя формат данных с сайта auto.ru. 

Еще раз взглянем на тестовый дата-сет после всех изменений:

In [32]:
df_test.head()

,body_type,brand,color,complectation,description,engine_vol,engine_power,equipment,fuel_type,mileage,model_date,model_name,doors_num,parsing_unixtime,production_date,transmission,vendor,num_owners,ownership,docs,drive,steering
0,LIFTBACK,SKODA,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2,105,"{""engine-proof"":true,""tinted-glass"":true,""airb...",GASOLINE,74000,2013,OCTAVIA,5,1603226273,2014,ROBOT,0,3,NaN,0,FORWARD_CONTROL,0
1,LIFTBACK,SKODA,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6,110,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",GASOLINE,60563,2017,OCTAVIA,5,1603226277,2017,MECHANICAL,0,1,NaN,0,FORWARD_CONTROL,0
2,LIFTBACK,SKODA,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8,152,"{""cruise-control"":true,""tinted-glass"":true,""es...",GASOLINE,88000,2013,SUPERB,5,1603226280,2014,ROBOT,0,1,NaN,0,FORWARD_CONTROL,0
3,LIFTBACK,SKODA,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6,110,"{""cruise-control"":true,""roller-blind-for-rear-...",GASOLINE,95000,2013,OCTAVIA,5,1603226284,2014,AUTOMATIC,0,1,NaN,0,FORWARD_CONTROL,0
4,LIFTBACK,SKODA,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8,152,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",GASOLINE,58536,2008,OCTAVIA,5,1603226288,2012,AUTOMATIC,0,1,NaN,0,FORWARD_CONTROL,0


И сохраним его для дальнейшей работы

In [33]:
df_test.to_csv('test_clean.csv', index=False)